In [2]:
###   Siamese network example
import torch
import torch.functional as F
import torch.nn as nn
import torchvision
import numpy as np

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
train_data = torchvision.datasets.CIFAR10(root = '../datasets/', train=True,
                                        transform=torchvision.transforms.ToTensor(),
                                        download=True)
test_data = torchvision.datasets.CIFAR10(root = '../datasets/', train=False,
                                        transform=torchvision.transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=16,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=16,
                                          shuffle=False)

Files already downloaded and verified


In [7]:
class Siamese(nn.Module):
    def __init__(self):
        super(Siamese, self).__init__()
        self.cnn1 = nn.Sequential(
                    nn.Conv2d(1, 96, kernel_size=11, stride=1),
                    nn.ReLU(inplace=True),
                    nn.LocalResponseNorm(5, alpha=0.0001,beta=0.75,k=2),
                    nn.MaxPool2d(3, stride=2),
            
                    nn.Conv2d(96, 256, kernel_size=5,stride=1,padding=2),
                    nn.ReLU(inplace=True),
                    nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2),
                    nn.MaxPool2d(3, stride=2),
                    nn.Dropout2d(p=0.3),

                    nn.Conv2d(256,384 , kernel_size=3,stride=1,padding=1),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(384,256 , kernel_size=3,stride=1,padding=1),
                    nn.ReLU(inplace=True),
                    nn.MaxPool2d(3, stride=2),
                    nn.Dropout2d(p=0.3)
                    )

        self.fc1 =  nn.Sequential(
                    nn.Linear(30976, 1024),
                    nn.ReLU(inplace=True),
                    nn.Dropout2d(p=0.5),
                    # Second Dense Layer
                    nn.Linear(1024, 128),
                    nn.ReLU(inplace=True),
                    nn.Linear(128,2)
        
                    ) 
        
    def forward_ind(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output
    
    def forward(self, x1, x2):
        # forward pass for input 1
        out1 = self.forward_ind(x1)
        # forward pass for input 2
        out2 = self.forward_ind(x2)
        return out1, out2


In [ ]:
class ContrastiveLoss(torch.nn.Module):

      def __init__(self, margin=2.0):
            super(ContrastiveLoss, self).__init__()
            self.margin = margin

      def forward(self, output1, output2, label):
            # Find the pairwise distance or eucledian distance of two output feature vectors
            euclidean_distance = F.pairwise_distance(output1, output2)
            # perform contrastive loss calculation with the distance
            loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
            (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

            return loss_contrastive